In [1]:
# remember to update these from functions from the package for development
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize
import random
import sys
import os
import math
import scipy.stats
from tabulate import tabulate
import time
import joblib


from confidenceset.random_field_generator import *
from confidenceset.confset import *
from confidenceset.plot import *
from confidenceset.test import *

%load_ext autoreload
%autoreload 2

## Simulations



In [ ]:
# create various simulation settings
from itertools import product

n_ls = [20, 40, 60, 80, 100]
fwhm_ls = [0, 2, 4, 6, 8]
#shape_ls = ['circular','ellipse','ramp']
shape_ls = ['ellipse','ramp']
noise_type_ls = ['gaussian','t']

# for 50*50, the actual fwhm_noise is half its input value
setting_df = pd.DataFrame(product(n_ls, [50],[50],fwhm_ls, shape_ls, [1, 10], noise_type_ls), columns=['n', 'w', 'h', 'fwhm_noise', 'shape', 'std', 'noise_type'])
#print(setting_df)

setting_df_resting = pd.DataFrame({'subsample_size': [20,40]})
print(setting_df_resting)

method_df1 = pd.DataFrame(product(['obs', 'res'], ['nonparametric'], ['t', 'z'], ['NA']), columns=['boot_data_type', 'boot_type', 'standardize', 'multiplier'])
method_df2 = pd.DataFrame(product(['res'], ['multiplier'], ['t', 'z'], ['r','g']), columns=['boot_data_type', 'boot_type', 'standardize', 'multiplier'])

method_df = pd.concat([method_df1, method_df2], ignore_index=True)

print(method_df)


In [ ]:
from SimuInf.scb import confband
from SimuInf.plotting import confset_plot, ls_plot
from SimuInf.simulation import scb_cover_rate_multiple

In [ ]:
# more simulations 
import joblib
simu_df_1to30 = scb_cover_rate_multiple(setting_df.iloc[:30], method_df, 
                      m_sim=1000, alpha=0.05,
                      m_boots=1000)
joblib.dump(simu_df_1to30, 'results/simu_df_1to30')

In [ ]:
simu_df_1to30 = joblib.load('results/simu_df_1to30')
simu_df_1to30

In [ ]:
# more simulations 
import joblib
simu_df_31to200 = scb_cover_rate_multiple(setting_df.iloc[30:], method_df, 
                      m_sim=1000, alpha=0.05,
                      m_boots=1000)
#joblib.dump(simu_df_31to200, 'results/simu_df_31to200')

In [ ]:
simu_df_31to200 = joblib.load('results/simu_df_31to200')
simu_df_31to200

In [ ]:
# more simulations 
import joblib
simu_df_201to400 = scb_cover_rate_multiple(setting_df2, method_df, 
                      m_sim=1000, alpha=0.05,
                      m_boots=1000)
joblib.dump(simu_df_201to400, 'results/simu_df_201to400')

In [ ]:
simu_df_201to400 = joblib.load('results/simu_df_201to400')
simu_df_201to400

In [31]:
simu_df_all = pd.concat([simu_df_1to30, simu_df_31to200, simu_df_201to400], ignore_index=True)
simu_df_all
simu_df_all.to_excel("simu_df_all.xlsx")  